In [15]:
from bs4 import BeautifulSoup
from requests import get
from unidecode import unidecode
import re

In [2]:
url = "http://www.imolaprogramma.it/imola-programma-2018/"

In [3]:
response = get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [46]:
program = soup.find('div', class_='et_pb_row et_pb_row_15')
day1 = program.find('div', class_='et_pb_column et_pb_column_1_2 et_pb_column_39')
day2 = program.find('div', class_='et_pb_column et_pb_column_1_2 et_pb_column_40')
program = [day1, day2]
program = [
    [unidecode(x.replace("–", "-")) 
     for x in [day.find('h3').text] + [x.text.strip() for x in day.find_all('p')] 
     if x] 
    for day in program]

In [47]:
print("\n".join([x for day in program for x in day ]))

Venerdi 16 Marzo
14.00 - 14.30 Saluti e apertura
Fabio Montefiori - LocalFocus
Davide Baroncini - Innovami
14:30 - 16:30 SANTERNO INNOVATION VALLEY
Roberto Cavina - Iprel Progetti Srl (Machine learning)
Claudio Guidi - Italianasoftware (Architetture distribuite a microservizi)
Alessandro Cillario - Cubbit (Cloud distributivo)
Leandro Rubbini - Exe.it (Data center Green)
Valeriano Ballardini - Special Video (Sistemi di visione)
Fabio Poli - Antreem (Service design)
16:30 - 17:00 Speech Tecnologie per la sicurezza dei lavoratori in trasferta
Marco Federzoni - TomTom Telematics
17:15 - 17:30 Il primo frutto di Imola Programma: una start up per il sociale
Andrea Padovano - Comprendiamo
17:30 - 18:30 DALL'INDUSTRY ALL'IMPRESA 4.0
Gildo Bosi - Sacmi
Matteo Masi - Cisco
Davide Dall'Osso - Cefla
Luca Cavina - SocialCities
Federica Minarelli - FAV
Andrea Bortolotti - Network Consulting
Maria Seralessandri - Imola Informatica
18:30 - 19:30 Data protection: scenario dopo il 25 maggio
Filippo Camp

In [61]:
time_regex = r"(?P<from>\d{1,2}\D{1}\d{2})\s*-\s*(?P<to>\d{1,2}\D{1}\d{2})\s*(?P<title>.*)"

In [68]:
speaker_regex = r"(?P<speaker>.+)\s*-\s*(?P<company>[a-zA-Z\s*]+)(\((?P<theme>.*[^\)])\))?"

In [69]:
date_regex = r"(?P<dayOfWeek>(?P<dayShort>Lun|Mar|Mer|Gio|Ven|Sab|Dom)\D*)\s(?P<day>\d*)\s(?P<month>(?P<monthShort>Gen|Feb|Mar|Apr|Mag|Giu|Lug|Ago|Set|Ott|Nov|Dic)\D*)"

In [70]:
re_list = [time_regex, speaker_regex]

In [71]:
pp = {}
re_count = 0
for day in program:
    for event in day:
#         r = re.compile(re_list[re_count])
        match = re.match(re_list[re_count], event)
        if not match:
            re_count = (re_count + 1) % len(re_list)
            match = re.match(re_list[re_count], event)
            #if match:
                
            #parts = match.groupdict()
            
            #re_count = (re_count + 1) % len(re_list)
        print(match, re_count)

None 1
<_sre.SRE_Match object; span=(0, 8), match='14.00 - '> 1
<_sre.SRE_Match object; span=(0, 29), match='Fabio Montefiori - LocalFocus'> 1
<_sre.SRE_Match object; span=(0, 27), match='Davide Baroncini - Innovami'> 1
<_sre.SRE_Match object; span=(0, 8), match='14:30 - '> 1
<_sre.SRE_Match object; span=(0, 54), match='Roberto Cavina - Iprel Progetti Srl (Machine lear> 1
<_sre.SRE_Match object; span=(0, 74), match='Claudio Guidi - Italianasoftware (Architetture di> 1
<_sre.SRE_Match object; span=(0, 49), match='Alessandro Cillario - Cubbit (Cloud distributivo)> 1
<_sre.SRE_Match object; span=(0, 21), match='Leandro Rubbini - Exe'> 1
<_sre.SRE_Match object; span=(0, 57), match='Valeriano Ballardini - Special Video (Sistemi di > 1
<_sre.SRE_Match object; span=(0, 37), match='Fabio Poli - Antreem (Service design)'> 1
<_sre.SRE_Match object; span=(0, 8), match='16:30 - '> 1
<_sre.SRE_Match object; span=(0, 35), match='Marco Federzoni - TomTom Telematics'> 1
<_sre.SRE_Match object; span=(0

In [ ]:
def format_event(ev_type, ev_dict):
    if ev_type == 0: # title
        pass
    elif ev_type == 1: # time
        pass
    else: # speaker
        pass
    

In [53]:
r"(?P<speaker>.+\S)\s*-\s*(?P<company>.+[^\(])(\((?P<theme>.+[^\)])\))"

'(?P<speaker>.+\\S)\\s*-\\s*(?P<company>.+[^\\(])(\\((?P<theme>.+[^\\)])\\))'

In [54]:
r"\s*(?P<speaker>.+\S)\s*-\s*(?P<company>.+\S)\s*"

'\\s*(?P<speaker>.+\\S)\\s*-\\s*(?P<company>.+\\S)\\s*'

In [105]:
def my_parse(event):
    if event.startswith("Eventi collaterali"):
        return
    
    try:  # orario
        int(event[0])
        orario_parts = parse_orario(event)
        print("orario", orario_parts)
    except ValueError:  # speaker
        is_talk = "(" in event
        speaker_parts = parse_speaker(event)
        print("speaker", len(speaker_parts), is_talk, speaker_parts)

In [106]:
def parse_orario(event):
    parts = re.split(r"[-]",event)
    more_parts = parts[1].split()
    splitted = [parts[0]] + [more_parts[0]] + [" ".join(more_parts[1:])]
    return [x.strip() for x in splitted]

def parse_speaker(event):
    topic_present = "(" in event
    parts = re.split(r"[-\(\)]",event)
    clean_parts = [x.strip() for x in parts if x]
    return clean_parts

In [107]:
for day in program:
    print(day[0])
    for event in day[1:]:
        my_parse(event)
    print()

Venerdi 16 Marzo
orario ['14.00', '14.30', 'Saluti e apertura']
speaker 2 False ['Fabio Montefiori', 'LocalFocus']
speaker 2 False ['Davide Baroncini', 'Innovami']
orario ['14:30', '16:30', 'SANTERNO INNOVATION VALLEY']
speaker 3 True ['Roberto Cavina', 'Iprel Progetti Srl', 'Machine learning']
speaker 3 True ['Claudio Guidi', 'Italianasoftware', 'Architetture distribuite a microservizi']
speaker 3 True ['Alessandro Cillario', 'Cubbit', 'Cloud distributivo']
speaker 3 True ['Leandro Rubbini', 'Exe.it', 'Data center Green']
speaker 3 True ['Valeriano Ballardini', 'Special Video', 'Sistemi di visione']
speaker 3 True ['Fabio Poli', 'Antreem', 'Service design']
orario ['16:30', '17:00', 'Speech Tecnologie per la sicurezza dei lavoratori in trasferta']
speaker 2 False ['Marco Federzoni', 'TomTom Telematics']
orario ['17:15', '17:30', 'Il primo frutto di Imola Programma: una start up per il sociale']
speaker 2 False ['Andrea Padovano', 'Comprendiamo']
orario ['17:30', '18:30', "DALL'INDUSTR